In [1]:
from ML_Dataset_Model import *
from ML_Functions import *
from Ariel_Functions import *

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import leastsq
import h5py
import os
from tqdm import tqdm
from time import time
import scipy
import torch
from torch import nn
from torch.utils.data import DataLoader
torch.set_default_dtype(torch.float64)

In [2]:
N= 21988


path = 'TrainingData'
path_np = 'np_data'

# test_matrix = to_observed_matrix(h5py.File( os.path.join( path,'SpectralData.hdf5'),'r') ,pd.read_csv( os.path.join( path,'AuxillaryTable.csv')))[:N]
# spec_tr = torch.Tensor(test_matrix[:,:,1])
# noise = torch.Tensor(test_matrix[:,:,2])
# aux_tr = torch.Tensor(pd.read_csv( os.path.join( path,'AuxillaryTable.csv')).drop(pd.read_csv( os.path.join( path,'AuxillaryTable.csv')).columns[:1],axis=1).values)[:N]
# quart_tr = torch.Tensor(load_Quartile_Table( os.path.join( path,'QuartilesTable.csv'))[:N])
# del test_matrix

# Loads the data using numpy files from path, comment the block and uncomment the previous block to use the training data in the original
# format, assuming AuxillaryTable,QuartilesTable, SpectralData are all in a folder named TrainingData



with np.load(os.path.join(path_np,'all_training.npz')) as data:
    spec_tr = torch.Tensor(data['spec_tr'])[:N]
    noise = torch.Tensor(data['noise'])[:N]
    aux_tr = torch.Tensor(data['aux_tr'])[:N]
    soft_tr = torch.Tensor(data['FM'])[:N]
    quart_tr = torch.Tensor(data['quart_tr'])


In [3]:
# Comment the next line and uncomment the convert_dist line to do the parametrize the distribution instead of using
# preparametrization, obtained parameters might be slightly different due to degeneracies.

param_tr = torch.tensor(np.load(os.path.join(path_np,'dist_params.npy')))
# param_tr = torch.Tensor(convert_dist(quart_tr,trace_path='TrainingData\\'+'Tracedata.hdf5'))

param_tr[:,0,:] = 1-  param_tr[:,0,:]

In [4]:
torch.manual_seed(0)
# Train_test splitter, tr_ratio determines training_samples to total samples ratio. Set to 1 before 
# training for the final submission

tr_ratio=1
uniq = torch.unique(aux_tr,dim=0)

ind  = torch.ones(aux_tr.shape[0])==1
for i in range(1,6):
    ind[quart_tr[:,1,i].argsort()[:int(21988*(1-tr_ratio))]] = False
    ind[quart_tr[:,1,i].argsort()[int(21988*tr_ratio):]] = False
    
print(ind.sum()/21988)

tensor(1.)


In [5]:
spec_tr,aux_tr,noise,Tr,std_mn = preprocessor(spec_tr,aux_tr,noise,ind=ind)
aux_std,aux_mn = std_mn
param_mn,param_std = param_tr[ind,:,:].mean(axis=0),param_tr[ind,:,:].std(axis=0)

In [6]:
train_data = Combined_Quart_Dataset(spectra=spec_tr[ind],aux=aux_tr[ind],quart=param_tr[ind][:,:,:],noise=noise[ind])
train_loader = DataLoader(train_data, batch_size=64,
                        shuffle=True, num_workers=0)

if not(ind.all()):
    test_data = Combined_Quart_Dataset(spectra=spec_tr[~ind],aux=aux_tr[~ind],quart=param_tr[~ind][:,:,:],noise=noise[~ind])

else:
    test_data=train_data
    
test_loader = DataLoader(test_data, batch_size=len(test_data),
                        shuffle=True, num_workers=0)


In [7]:
model = Parameter_ML(Dropout=0.2)
# Model Parameter_SML is not used in the rest of the code, however it was defined in the run where final submission
# was created and thus, will be kept to make sure cdf stays reproducible
model1 =  Parameter_SML(Dropout=0.2)

In [8]:
# Loss function is an approximation of the EMD
loss_fn = lambda y,yhat: (yhat[:,0,:]-y[:,0,:])**2*5+(1-y[:,0,:])**2*torch.abs(y[:,1,:]-yhat[:,1,:])**2+(
    y[:,0,:])**2* ((y[:,2,:]-yhat[:,2,:])**2+yhat[:,3,:]+y[:,3,:]-2*torch.sqrt(y[:,3,:]*yhat[:,3,:]))

params={'model':model ,
      'loss_fn':loss_fn,
      'train_dl':train_loader,
      'val_dl':test_loader,
      'device':'cpu', 'optimizer':torch.optim.Adam(model.parameters(),8e-4)}

a=train(**params,epochs=np.array([10,15,30]),lrs=np.array([8e-4,4e-4,2e-4]))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/344 [00:00<?, ?it/s]

  0%|          | 0/344 [00:00<?, ?it/s]

...

  0%|          | 0/344 [00:00<?, ?it/s]

  0%|          | 0/344 [00:00<?, ?it/s]

  0%|          | 0/344 [00:00<?, ?it/s]

  0%|          | 0/344 [00:00<?, ?it/s]

Time total: 359.32 sec, Best Val. Loss:35.527040568735615, Best of Worst of:0.0


In [9]:
# plt.plot(a['history']['loss'][10:],label='tr')
# plt.plot(a['history']['val_loss'][10:],label='val')
# plt.legend()

In [12]:
# Load the final data 

path_final  = 'final'

test_matrix = to_observed_matrix(h5py.File(os.path.join(path_final,'SpectralData.hdf5'),'r'),
                                           pd.read_csv(os.path.join(path_final,'AuxillaryTable.csv')))
spec_test = torch.Tensor(test_matrix[:,:,1])
noise_test = torch.Tensor(test_matrix[:,:,2])
aux_test = torch.Tensor(pd.read_csv(os.path.join(path_final,'AuxillaryTable.csv')).drop(pd.read_csv(os.path.join(path_final,'AuxillaryTable.csv')).columns[:2],axis=1).values)

In [16]:
# Load the data challenge test data (not the final data)

with np.load(os.path.join(path_np, 'all_testing.npz') ) as data:
    spec_test = torch.Tensor(data['spec_test'])
    noise_test = torch.Tensor(data['noise_test'])
    aux_test = torch.Tensor(data['aux_test'])

In [17]:
spec_test,aux_test,noise_test,T,std_mn =  preprocessor(spec_test,aux_test,noise_test,std_mn=std_mn)

In [18]:
y = np.zeros((spec_test.shape[0],3,6))
model.eval()

params_test = model(spec_test,aux_test,noise_test).detach().numpy()
temp = np.linspace(-12,-1,10000)

y[:,0,0] = T*0.9865
y[:,1,0] = T
y[:,2,0] = T*1.016

for i in tqdm(range(y.shape[0])):
    for j in range(1,6):
        
        tempp = dist_func(params_test[i,:,j-1],temp)
        y[i,0,j] = temp[ np.abs(tempp-0.16).argsort()[0]]
        y[i,1,j] = temp[ np.abs(tempp-0.5).argsort()[0]]
        y[i,2,j] = temp[ np.abs(tempp-0.84).argsort()[0]]

y[np.logical_and(y>-1,y<0)]=-1
y.sort(axis=1)
y[y<-12]=-12


100%|███████████████████████████████████████████████████████████████████████████████| 800/800 [00:03<00:00, 215.17it/s]


In [19]:
# yp = load_Quartile_Table('LT_Submission_984LT.csv')
# light_track_metric(yp[:,:,:],y[:,:,:])

score is: 1000.0


1000.0

In [17]:
# Creates the distribution and saves it in RT_submission.csv
N=4998

y_dist = np.zeros((N,spec_test.shape[0],6))

mean = y[:,1,:]
std1  = (y[:,1,:]-y[:,0,:])
std2  = (y[:,2,:]-y[:,1,:])


y_dist[:int(N/2),:,0] = mean[:,0] - np.abs(np.random.normal(0, np.broadcast_to(std1[:,0],(int(N/2),spec_test.shape[0])) ))
y_dist[int(N/2):,:,0] = mean[:,0] + np.abs(np.random.normal(0, np.broadcast_to(std2[:,0],(int(N/2),spec_test.shape[0])) ))

y_dist[:,:,1:] = sample_dist(params_test)

for i in range(y_dist.shape[1]):
    for j in range(1,6):
        np.random.shuffle(y_dist[:,i,j])

y_dist = y_dist.swapaxes(0,1)

RT_submission = to_regular_track_format(y_dist, 
   np.ones(y_dist.shape[:-1])*1/N)
            

  0%|          | 0/800 [00:00<?, ?it/s]

In [17]:
np.round(y[:,:,1:].max(axis=0).T,3)

array([[-2.198, -2.193, -2.187],
       [-4.107, -3.999, -3.763],
       [-2.858, -2.802, -2.746],
       [-1.737, -1.7  , -1.612],
       [-3.902, -3.883, -3.865]])

In [ ]:
np.round(y[:,:,1:].min(axis=0).T,3)

In [ ]:
np.round(quart[:,:,1:].max(axis=0).values.T,3)

In [ ]:
np.round(quart[:,:,1:].min(axis=0).values.T,3)